In [1]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.datasets import load_wine
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [2]:
# Load the iris dataset and split into training and testing sets
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.20, random_state=46)


In [3]:
print("X train set shpae - ",X_train.shape)
print("y train set shpae - ",y_train.shape)
print("X test set shpae - ",X_test.shape)
print("y test set shpae - ",y_test.shape)

X train set shpae -  (142, 13)
y train set shpae -  (142,)
X test set shpae -  (36, 13)
y test set shpae -  (36,)


In [4]:
# Create the base models
knn = KNeighborsClassifier(n_neighbors=90)
rf = RandomForestClassifier(n_estimators=100)
dt = DecisionTreeClassifier(max_depth=5)
svc = SVC(C=1, gamma=0.1, kernel='linear')

In [5]:
# Step 2: Define the number of folds for cross-validation
num_folds = 7

In [6]:
# Step 3: Train several base models on the K-1 folds and use them to make predictions on the Kth fold
kf = KFold(n_splits=num_folds)
base_models = [rf, dt, knn]
base_predictions = np.zeros((X_train.shape[0], len(base_models)))

In [7]:
for i, model in enumerate(base_models):
    for train_idx, val_idx in kf.split(X_train):
        X_train_fold, y_train_fold = X_train[train_idx], y_train[train_idx]
        X_val_fold, y_val_fold = X_train[val_idx], y_train[val_idx]
        model.fit(X_train_fold, y_train_fold)
        base_predictions[val_idx, i] = model.predict(X_val_fold)
        
base_predictions

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

array([[0., 0., 0.],
       [1., 1., 2.],
       [0., 0., 2.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 2.],
       [1., 1., 1.],
       [2., 2., 1.],
       [2., 2., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [2., 2., 1.],
       [0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 2.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 1., 1.],
       [2., 2., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 1.],
       [2., 2., 1.],
       [2., 2., 1.],
       [0., 0., 2.],
       [1., 1., 1.],
       [2., 2., 1.],
       [1., 1., 2.],
       [2., 2., 1.],
       [0., 0., 0.],
       [2., 2., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [2., 2

In [8]:
# Step 4: Train a meta-model on the predictions from step 3
meta_model = svc
meta_model.fit(base_predictions, y_train)

SVC(C=1, gamma=0.1, kernel='linear')

In [9]:
# Step 5: Repeat steps 3-4 for all K folds
meta_predictions = np.zeros((X_test.shape[0], len(base_models)))

In [10]:
for i, model in enumerate(base_models):
    model.fit(X_train, y_train)
    meta_predictions[:, i] = model.predict(X_test)

meta_predictions

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array([[2., 2., 1.],
       [1., 1., 1.],
       [0., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [1., 1., 1.],
       [2., 2., 2.],
       [2., 2., 1.],
       [2., 2., 2.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [2., 2., 1.],
       [0., 0., 0.],
       [0., 0., 2.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 2.],
       [0., 0., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 1., 0.],
       [2., 2., 2.],
       [1., 1., 1.],
       [2., 2., 1.],
       [2., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.],
       [2., 2., 1.],
       [1., 1., 1.],
       [2., 2., 1.]])

In [11]:
# Step 6: Use the meta-model to make predictions on the testing data
final_predictions = meta_model.predict(meta_predictions)
final_predictions

array([2, 1, 0, 1, 0, 1, 2, 2, 2, 1, 0, 0, 1, 1, 2, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 2, 1, 2, 2, 0, 0, 0, 1, 2, 1, 2])

In [12]:
# Step 7: Evaluate the performance of the stacking model
accuracy = accuracy_score(y_test, final_predictions)

In [14]:
print("Stacking Essemble Model Accuracy = ",accuracy)

Stacking Essemble Model Accuracy =  0.9444444444444444


# Much Effecient 